## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_copy import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
# I was not able to load WeatherPy_Database from the Weather_Database folder so I had so also save it in the Vacation Search folder 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Vaini,TO,2020-10-15 19:01:35,-21.20,-175.20,73.40,83,90,8.05,overcast clouds
1,1,Hofn,IS,2020-10-15 19:05:37,64.25,-15.21,45.03,78,0,5.41,clear sky
2,2,Matara,LK,2020-10-15 19:02:14,5.95,80.54,79.45,84,95,11.99,moderate rain
3,3,Lima,PE,2020-10-15 19:05:17,-12.04,-77.03,69.80,64,20,10.29,few clouds
4,4,Rawson,AR,2020-10-15 19:05:37,-43.30,-65.10,77.56,19,0,17.16,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
2,2,Matara,LK,2020-10-15 19:02:14,5.95,80.54,79.45,84,95,11.99,moderate rain
4,4,Rawson,AR,2020-10-15 19:05:37,-43.30,-65.10,77.56,19,0,17.16,clear sky
8,8,Guerrero Negro,MX,2020-10-15 19:05:38,27.98,-114.06,88.72,42,97,4.12,overcast clouds
9,9,Srandakan,ID,2020-10-15 19:05:38,-7.94,110.25,77.02,88,100,5.57,light rain
14,14,Atuona,PF,2020-10-15 19:05:38,-9.80,-139.03,78.58,71,0,20.54,clear sky
16,16,Bengkulu,ID,2020-10-15 19:01:50,-3.80,102.27,76.82,83,100,2.44,overcast clouds
23,23,Sung Noen,TH,2020-10-15 19:05:39,14.90,101.82,75.07,93,99,3.44,overcast clouds
25,25,Lake Charles,US,2020-10-15 19:05:40,30.21,-93.20,84.99,62,75,13.87,broken clouds
26,26,Batticaloa,LK,2020-10-15 19:03:02,7.71,81.69,80.11,73,92,10.89,moderate rain
31,31,Sinnamary,GF,2020-10-15 19:02:56,5.38,-52.95,85.86,61,50,13.22,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


Unnamed: 0         0
City               0
Country            2
Date               0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Matara,LK,79.45,moderate rain,5.95,80.54,
4,Rawson,AR,77.56,clear sky,-43.30,-65.10,
8,Guerrero Negro,MX,88.72,overcast clouds,27.98,-114.06,
9,Srandakan,ID,77.02,light rain,-7.94,110.25,
14,Atuona,PF,78.58,clear sky,-9.80,-139.03,
16,Bengkulu,ID,76.82,overcast clouds,-3.80,102.27,
23,Sung Noen,TH,75.07,overcast clouds,14.90,101.82,
25,Lake Charles,US,84.99,broken clouds,30.21,-93.20,
26,Batticaloa,LK,80.11,moderate rain,7.71,81.69,
31,Sinnamary,GF,85.86,scattered clouds,5.38,-52.95,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [15]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Matara,LK,79.45,moderate rain,5.95,80.54,Amaloh Boutique Resort
4,Rawson,AR,77.56,clear sky,-43.30,-65.10,Hotel Punta León
8,Guerrero Negro,MX,88.72,overcast clouds,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
9,Srandakan,ID,77.02,light rain,-7.94,110.25,Puri Brata
14,Atuona,PF,78.58,clear sky,-9.80,-139.03,Villa Enata
16,Bengkulu,ID,76.82,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
23,Sung Noen,TH,75.07,overcast clouds,14.90,101.82,ฉัตรทิพย์ รีสอร์ท
25,Lake Charles,US,84.99,broken clouds,30.21,-93.20,La Quinta Inn & Suites by Wyndham Lake Charles...
26,Batticaloa,LK,80.11,moderate rain,7.71,81.69,Grand Star Guest House
31,Sinnamary,GF,85.86,scattered clouds,5.38,-52.95,ALETHEIA Appartement Sinnamary


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))